In [1]:
import importlib
import sys
import torch
from torch.utils.data import random_split
import random
from collections.abc import Iterator
from tqdm.notebook import tqdm
from Levenshtein import distance
import collections
import pickle
import warnings
import pandas as pd
import matplotlib.pyplot as plt
import CRPS.CRPS as pscore
import os


sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')


from model.dropout_uncertainty_enc_dec_LSTM_new_dec.dropout_uncertainty_model import DropoutUncertaintyEncoderDecoderLSTM

In [2]:
#load model
file_path_model = '../../training_new_dec/Helpdesk/Helpdesk_full_no_grad_norm_new_1.pkl'
model = DropoutUncertaintyEncoderDecoderLSTM.load(file_path_model, dropout=0.1)




# Load the dataset
file_path_data_set = '../../../../encoded_data/helpdesk_all_5_test.pkl'
bpic_17_test_dataset = torch.load(file_path_data_set, weights_only=False)

Data set categories:  ([('Activity', 16, {'Assign seriousness': 1, 'Closed': 2, 'Create SW anomaly': 3, 'DUPLICATE': 4, 'EOS': 5, 'INVALID': 6, 'Insert ticket': 7, 'RESOLVED': 8, 'Require upgrade': 9, 'Resolve SW anomaly': 10, 'Resolve ticket': 11, 'Schedule intervention': 12, 'Take in charge ticket': 13, 'VERIFIED': 14, 'Wait': 15}), ('Resource', 24, {'EOS': 1, 'Value 1': 2, 'Value 10': 3, 'Value 11': 4, 'Value 12': 5, 'Value 13': 6, 'Value 14': 7, 'Value 15': 8, 'Value 16': 9, 'Value 17': 10, 'Value 18': 11, 'Value 19': 12, 'Value 2': 13, 'Value 20': 14, 'Value 21': 15, 'Value 22': 16, 'Value 3': 17, 'Value 4': 18, 'Value 5': 19, 'Value 6': 20, 'Value 7': 21, 'Value 8': 22, 'Value 9': 23}), ('VariantIndex', 175, {'1.0': 1, '10.0': 2, '100.0': 3, '103.0': 4, '104.0': 5, '107.0': 6, '109.0': 7, '11.0': 8, '110.0': 9, '112.0': 10, '113.0': 11, '114.0': 12, '115.0': 13, '117.0': 14, '118.0': 15, '12.0': 16, '120.0': 17, '122.0': 18, '123.0': 19, '124.0': 20, '125.0': 21, '126.0': 22, '12

In [3]:
import evaluation.probabilistic_evaluation
importlib.reload(evaluation.probabilistic_evaluation)
from evaluation.probabilistic_evaluation import ProbabilisticEvaluation

new_eval = ProbabilisticEvaluation(model, bpic_17_test_dataset,
                                   concept_name='Activity',
                                   num_processes=16,
                                   #growing_num_values = [],
                                   growing_num_values = ['case_elapsed_time'],
                                   samples_per_case = 1000,
                                   sample_argmax = False,
                                   use_variance_cat = True,
                                   use_variance_num = True,
                                   )


In [4]:
def save_chunk(results, i):
    chunk_number = (i + 1)
    filename = os.path.join(output_dir, f'results_part_{chunk_number:03d}.pkl')
    with open(filename, 'wb') as f:
        pickle.dump(results, f)
    print(f"Saved {len(results)} results to {filename}")


output_dir = '../../../../../evaluation_results/Helpdesk/no_gn'
save_every = 50
results = {}
for i, (case_name, prefix_len, prefix, predicted_suffixes, suffix, mean_prediction) in enumerate(new_eval.evaluate_multi_processing(random_order=True)):
    print(case_name, prefix_len)
    assert((case_name, prefix_len) not in results)
    results[(case_name, prefix_len)] = (prefix, suffix, mean_prediction, predicted_suffixes)
    print(prefix_len, len(suffix))
    if (i + 1) % save_every == 0:
        save_chunk(results, i)
        results = {}

if len(results):
    save_chunk(results, i)

  0%|          | 0/535 [00:00<?, ?it/s]

Case 3228 3
3 1
Case 2603 3
3 1
Case 1660 3
3 1
Case 205 3
3 1
Case 1032 3
3 1
Case 1032 1
1 3
Case 205 2
2 2
Case 2603 1
1 3
Case 1032 2
2 2
Case 3228 2
2 2
Case 2603 2
2 2
Case 1660 2
2 2
Case 1660 1
1 3
Case 205 1
1 3
Case 3228 1
1 3
Case 3126 1
1 3
Case 3037 3
3 1
Case 3037 2
2 2
Case 3243 3
3 7
Case 3126 3
3 1
Case 3037 1
1 3
Case 1700 2
2 3
Case 1700 4
4 1
Case 3243 4
4 6
Case 3126 2
2 2
Case 3243 5
5 5
Case 1700 3
3 2
Case 3243 7
7 3
Case 3243 2
2 8
Case 3243 6
6 4
Case 3243 1
1 9
Case 1700 1
1 4
Case 3243 9
9 1
Case 3243 8
8 2
Case 1724 1
1 4
Case 1724 4
4 1
Case 1724 2
2 3
Case 2065 2
2 2
Case 1724 3
3 2
Case 2065 1
1 3
Case 2275 2
2 3
Case 2065 3
3 1
Case 2275 3
3 2
Case 2275 1
1 4
Case 2270 1
1 4
Case 2270 2
2 3
Case 2270 3
3 2
Case 2275 4
4 1
Case 2270 4
4 1
Case 171 1
1 3
Saved 50 results to ../../../../../evaluation_results/Helpdesk/no_gn/results_part_050.pkl
Case 171 2
2 2
Case 171 3
3 1
Case 3276 3
3 1
Case 3276 2
2 2
Case 2708 3
3 1
Case 2708 1
1 3
Case 2853 1
1 3
Case